In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from DataFields import DateReportedFields
from ProjectFunctions import one_hot_encode_vascular_problems, convert_date_to_binary
from ProjectFunctions import count_na_in_dataframe, drop_rows_with_na_greater_than

### Prepare the dataset for training

In [2]:
df_diagnosed = pd.read_csv("diagnosed_imputed.csv")
df_undiagnosed = pd.read_csv("undiagnosed.csv")

df_merged = pd.concat([df_diagnosed, df_undiagnosed], ignore_index=True, sort=False)

df_merged = one_hot_encode_vascular_problems(df_merged)
df_merged = df_merged.drop(columns=["Education"]) # columns to exclude, if wanted to 

In [8]:
df_merged.describe()

,Vascular Dementia Report Date,Birth Year,Sex,Primary Hypertension,Secondary Hypertension,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,...,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Stroke Report Date,Heart Attack,Angina,Stroke,High Blood Pressure
count,2036.0,4272.000000,4272.000000,2036.000000,2036.000000,4261.000000,4257.000000,4258.000000,4267.000000,2036.000000,...,4190.000000,4219.000000,4100.000000,4100.000000,4100.000000,0.0,4272.000000,4272.000000,4272.000000,4272.000000
mean,1.0,1943.189841,0.533474,0.789784,0.002947,27.976433,0.639584,0.650526,3.066384,0.312377,...,1.390953,7.143009,82.094686,146.648569,70.605212,NaN,0.051498,0.036517,0.036985,0.169007
std,0.0,4.902158,0.498937,0.407562,0.054219,4.770852,0.660895,0.475869,1.632783,0.463577,...,0.668537,2.029500,10.779522,20.269976,12.393332,NaN,0.221037,0.187594,0.188747,0.374802
min,1.0,1937.000000,0.000000,0.000000,0.000000,15.525600,-0.181990,0.000000,1.000000,0.000000,...,0.227000,0.980000,46.000000,78.000000,35.000000,NaN,0.000000,0.000000,0.000000,0.000000
25%,1.0,1940.000000,0.000000,1.000000,0.000000,24.778100,0.000000,0.000000,2.000000,0.000000,...,1.005250,5.880000,75.000000,133.000000,62.000000,NaN,0.000000,0.000000,0.000000,0.000000
50%,1.0,1942.000000,1.000000,1.000000,0.000000,27.324600,1.000000,1.000000,3.000000,0.000000,...,1.310000,6.900000,82.000000,146.000000,69.000000,NaN,0.000000,0.000000,0.000000,0.000000
75%,1.0,1945.000000,1.000000,1.000000,0.000000,30.514900,1.000000,1.000000,4.000000,1.000000,...,1.685000,8.195000,89.000000,159.000000,77.320571,NaN,0.000000,0.000000,0.000000,0.000000
max,1.0,1968.000000,1.000000,1.000000,1.000000,58.260900,2.000000,1.000000,6.000000,1.000000,...,25.278000,46.600000,132.000000,241.000000,169.000000,NaN,1.000000,1.000000,1.000000,1.000000


In [9]:
count_na_in_dataframe(df_merged)

Stroke Report Date: 4272 NA rows
Primary Hypertension: 2236 NA rows
Secondary Hypertension: 2236 NA rows
Juvenile Arthritis: 2236 NA rows
Vascular Dementia Report Date: 2236 NA rows
Thyrotoxicosis (Grave's disease): 2236 NA rows
Sjogren Disease (M35): 2236 NA rows
Other Rheumatoid Arthritis: 2236 NA rows
Seropositive Rheumatoid Arthritis: 2236 NA rows
Other Arthritis: 2236 NA rows
Crohn's disease: 2236 NA rows
Multiple Sclerosis: 2236 NA rows
Psoriatic and enteropathic arthropathies: 2236 NA rows
Myasthenia gravis: 2236 NA rows
Ulcerative Colitis: 2236 NA rows
B12 deficiency anaemia: 2236 NA rows
Report of stroke: 2236 NA rows
Diagnosed with Coeliac disease: 1620 NA rows
Lipoprotein A: 533 NA rows
Direct bilirubin: 385 NA rows
Testosterone: 315 NA rows
SHBG: 249 NA rows
Apolipoprotein A: 234 NA rows
Phosphate: 232 NA rows
Glucose: 232 NA rows
Calcium: 232 NA rows
Total protein: 230 NA rows
HDL cholesterol: 230 NA rows
Albumin: 229 NA rows
Vitamin D: 177 NA rows
Pulse Rate at Blood Pres

In [4]:
X = df_merged.drop('Vascular Dementia Report Date', axis=1).values  # Features
y = df_merged['Vascular Dementia Report Date'].values  # Target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

/home/michaelpo2@mta.ac.il/miniconda3/envs/bio-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/michaelpo2@mta.ac.il/miniconda3/envs/bio-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/michaelpo2@mta.ac.il/miniconda3/envs/bio-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


### Define the neural network

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.layer2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.sigmoid(self.layer3(x))
        return x

In [6]:
# Initialize the model
input_size = X_train.shape[1]
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

# Convert the data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [7]:
# Training loop
epochs = 500
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()

    output_train = model(X_train_tensor)
    loss = criterion(output_train, y_train_tensor)

    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    predicted_train = (output_train > 0.5).float()
    train_accuracy = accuracy_score(y_train, predicted_train.numpy())

    # Evaluate on test batch
    model.eval()
    with torch.no_grad():
        output_test = model(X_test_tensor)
        predicted_test = (output_test > 0.5).float()
        test_accuracy = accuracy_score(y_test, predicted_test.numpy())

    if (epoch + 1) % (epochs // 10) == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item():.4f}, '
              f'Training Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')


RuntimeError: all elements of input should be between 0 and 1

### Evaluate the model performance

#### Model ROC-AUC curve

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)

In [ ]:
y_pred_prob = y_pred.numpy().flatten()
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

#### Confusion matrix

In [ ]:
y_pred_labels = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred_labels)

plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

### Feature importance extraction using SHAPley

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

explainer = shap.GradientExplainer(model, X_test_tensor)

# Compute SHAP values
shap_values = explainer.shap_values(X_test_tensor)

feature_names = df.drop(columns=["Has Vascular Dementia"]).columns.to_list()

# plot summary of feature importance
shap_values = shap_values.squeeze()  # Remove unnecessary dimensions
shap.summary_plot(shap_values, X_test, feature_names=feature_names)